<a href="https://colab.research.google.com/github/vsoos/colab/blob/main/Copy_of_EX4_notebook2_moviesdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the needed TF-recommenders package

In [62]:
!pip uninstall -q tensorflow-recommenders
!pip install -q tensorflow-recommenders --no-deps
!pip uninstall -q tensorflow
!pip install -q tensorflow==2.15.0

Proceed (Y/n)? Y

Proceed (Y/n)? Y
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.15.0 which is incompatible.


In [63]:
import string
import re
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
from wordcloud import WordCloud
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Get all the dataframes from Google Drive

In [64]:
folder_path = "/content/drive/MyDrive/cloudcomputing2023_VincenzinaSoos/ex4/4_notebook2_gpu"
os.chdir(folder_path)

In [65]:
credits = pd.read_csv('data/credits.csv')
keywords = pd.read_csv('data/keywords.csv')
movies = pd.read_csv('data/movies_metadata.csv').\
                     drop(['belongs_to_collection', 'homepage', 'imdb_id', 'poster_path', 'status', 'title', 'video'], axis=1).\
                     drop([19730, 29503, 35587]) # Incorrect data type

movies['id'] = movies['id'].astype('int64')

df = movies.merge(keywords, on='id').\
    merge(credits, on='id')

df['original_language'] = df['original_language'].fillna('')
df['runtime'] = df['runtime'].fillna(0)
df['tagline'] = df['tagline'].fillna('')

df.dropna(inplace=True)

In [66]:
df.head()

,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,keywords,cast,crew
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",,7.7,5415.0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,6.9,2413.0,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Still Yelling. Still Fighting. Still Ready for...,6.5,92.0,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Friends are the people who let you be yourself...,6.1,34.0,"[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Just When His World Is Back To Normal... He's ...,5.7,173.0,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [67]:
sorted_df = df.sort_values(by='release_date', ascending=False)

In [68]:
# newest movies in the dataset are practically from 2018
# exception: Avatar 2 => not released in this point, no rating data etc.
# we could remove it? row id == 26774, movie id = 76600
sorted_df.head()

,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,keywords,cast,crew
26774,False,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",76600,en,Avatar 2,A sequel to Avatar (2009).,6.020055,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2020-12-16,0.0,0.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",,0.0,58.0,"[{'id': 9663, 'name': 'sequel'}, {'id': 10158,...","[{'cast_id': 47, 'character': 'Ronal', 'credit...","[{'credit_id': '52fe4943c3a368484e122b45', 'de..."
40053,False,12000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",299782,en,The Other Side of the Wind,"Orson Welles' unfinished masterpiece, restored...",0.238154,"[{'name': ""Les Films de l'Astrophore"", 'id': 7...","[{'iso_3166_1': 'IR', 'name': 'Iran'}, {'iso_3...",2018-12-31,0.0,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",,0.0,1.0,"[{'id': 1228, 'name': '1970s'}, {'id': 4610, '...","[{'cast_id': 21, 'character': 'J.J. Jake Hanna...","[{'credit_id': '54516d4b0e0a2601d8004b8a', 'de..."
30612,False,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",38700,en,Bad Boys for Life,The continuing adventures of Miami detectives ...,2.178546,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",2018-11-07,0.0,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",,0.0,12.0,"[{'id': 4414, 'name': 'adventure'}, {'id': 966...","[{'cast_id': 0, 'character': 'Detective Mike L...","[{'credit_id': '585fcddd92514115cd01d8c8', 'de..."
39299,False,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",332283,en,Mary Shelley,The love affair between poet Percy Shelley and...,3.328261,"[{'name': 'Parallel Films', 'id': 5122}, {'nam...","[{'iso_3166_1': 'IE', 'name': 'Ireland'}, {'is...",2018-04-25,0.0,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",,0.0,1.0,"[{'id': 187056, 'name': 'woman director'}]","[{'cast_id': 2, 'character': 'Mary Shelley', '...","[{'credit_id': '588f287ac3a36860d1005f7a', 'de..."
45700,False,0,"[{'id': 18, 'name': 'Drama'}]",412059,en,Mobile Homes,"In forgotten towns along the American border, ...",0.155147,[],"[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",2018-04-04,0.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",,0.0,1.0,[],"[{'cast_id': 0, 'character': 'Ali', 'credit_id...","[{'credit_id': '57b78978c3a36823a60029a8', 'de..."


In [69]:
df['release_date'] = pd.to_datetime(df['release_date'])
df['budget'] = df['budget'].astype('float64')
df['popularity'] = df['popularity'].astype('float64')

Loading up the ratings...

In [70]:
# the rating system between 0 and 5 (the common 5-star system)
# ratings_df['rating'].min()

In [71]:
# create a list of movie names, 10-20
# preferably some of them being bad and average movies
# have a list of movies, good movies, average movies, bad movies (based on YOUR opinion)
# at least 15-20, you can have more too
# rate each movie between 0 - 5, half-decimals are allowed (for example: 3.5)
my_movies = {
    "Amadeus": 5,
    "Terminator 2: Judgment Day": 5,
    "The Lord of the Rings: The Return of the King": 5,
    "Batman Begins": 4.5,
    "The Men Who Stare at Goats": 4,
    "The Grand Budapest Hotel": 4.5,
    "Eternal Sunshine of the Spotless Mind": 4,
    "Inception": 4,
    "Cloud Atlas": 4.5,
    "Coco": 5,
    "The Mask|1994": 4,
    "Ace Ventura: Pet Detective": 4,
    "Blade Runner": 5,
    "The Void|2016": 4,

    #"Iron Man|2008": 3.5,
    #"The Hobbit: The Battle of the Five Armies": 3.5,
    #"The Angry Birds Movie": 3,
    #"Jurassic Park": 3,
    #"Airheads": 3,
    #"Last Action Hero": 3,
    #"The Godfather": 3,

    #"The Blair Witch Project": 2.5,
    #"Space Jam": 2.5,
    #"Ace Ventura: When Nature Calls": 3.5,
    #"Liar Liar": 3,
    #"Star Wars: Episode I - The Phantom Menace": 3,


    #"Hook": 2,
    #"The Flintstones": 2,
    #"Batman & Robin": 1,
    #"Sharknado": 1,
    #"Police Academy 6: City Under Siege": 1,
    #"Billy Madison": 1,
    #"Red Scorpion": 1.5,
    #"GoodFellas": 2,
    #"Wayne's World": 2,
    #"Scream": 1,
    #"Godzilla|1998": 2, # the 1998 movie
    #"Son of the Mask": 1,
    #"The Hunchback of Notre Dame II": 1,
    #"Street Fighter": 1.5

}

# how many movies in total now...
print(len(my_movies))

14


In [72]:
# searching a movie by id
df[df["id"] == 378018]

,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,keywords,cast,crew
43944,False,0.0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 27, '...",378018,en,The Void,"In the middle of a routine patrol, officer Dan...",9.082794,"[{'name': '120dB Films', 'id': 5113}, {'name':...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2016-10-21,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",There Is a Hell. This Is Worse.,5.8,207.0,"[{'id': 428, 'name': 'nurse'}, {'id': 1852, 'n...","[{'cast_id': 5, 'character': 'Daniel Carter', ...","[{'credit_id': '58ab0b4592514141e2004935', 'de..."


In [73]:
# searching a movie by exact title
df[df["original_title"] == "Star Wars: Episode I - The Phantom Menace"]


,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,keywords,cast,crew
2534,False,115000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",1893,en,Star Wars: Episode I - The Phantom Menace,"Anakin Skywalker, a young slave strong with th...",15.649091,"[{'name': 'Lucasfilm', 'id': 1}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1999-05-19,924317558.0,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Every generation has a legend. Every journey h...,6.4,4526.0,"[{'id': 530, 'name': 'prophecy'}, {'id': 1399,...","[{'cast_id': 8, 'character': 'Qui-Gon Jinn', '...","[{'credit_id': '52fe431ec3a36847f803bc9d', 'de..."


In [74]:
# searching with a partial movie name
df[df["original_title"].str.contains("Goodfellas", case=False)]
# df[df['original_title'] == ""]]

,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,keywords,cast,crew
1191,False,25000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",769,en,GoodFellas,"The true story of Henry Hill, a half-Irish, ha...",15.424092,"[{'name': 'Winkler Films', 'id': 8880}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1990-09-12,46836394.0,145.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Three Decades of Life in the Mafia.,8.2,3211.0,"[{'id': 378, 'name': 'prison'}, {'id': 818, 'n...","[{'cast_id': 16, 'character': 'James Conway', ...","[{'credit_id': '52fe4274c3a36847f801fcc3', 'de..."


In [75]:
# in order to create rows of new pandas data in a DataFrame
# we need a big dictionary, that has the same keys as the DataFrame (userId, movieId, rating, date)
# the actual data has to be in lists: list of userId's, list of movieId's etc.
# create empty lists that we fill later with actual data
# all lists have to be the exact same length. in this case, 40 movies => 40 values in EACH list
# we can repeat numbers in a list by using this syntax => [e] * n
userIds_list = [672] * len(my_movies)

# we have to fill these in the loop below
movieIds_list = []
ratings_list = []
dates_list = []

# unix epoch of the date => 2022-11-10
starting_date_timestamp = 1668067820
one_day_duration = 86400

# idea is this:

# before processing => randomize the order of the my_movie-dictionary JUST IN CASE

# go (for-loop) through every movie in the my_movies list
for movie_name in my_movies:
  # one movie at a time has a name, and a rating (my own rating)
  movie_rating = my_movies[movie_name]

  # get the current index from a helper list => the key values from the dictionary
  # starts from 0, ends depending on how many movies (40 movies => last index 39)
  movie_counter = list(my_movies.keys()).index(movie_name)

  movie_watch_date = starting_date_timestamp - (movie_counter * one_day_duration * 21)
  # look up for the movie with this name
  # find the matching movie id in the dataset
  # if the movie has a pipe sign in the name "|"
  if "|" not in movie_name:
    search_result = df[df["original_title"] == movie_name]['id']

    # check that we only have one movie with this name
    if len(search_result.index) == 1:
      movie_id = search_result.astype(int).item()
      movieIds_list.append(movie_id)
      ratings_list.append(movie_rating)
      dates_list.append(movie_watch_date)
    elif len(search_result.index) == 0:
      print("------------------------------")
      print(f"The following movie doesn't exist in the dataset (check exact spelling): {movie_name}")
      print("------------------------------")
    else:
      # there are multiple releases with the same year
      # the user has to define which year to use
      print("------------------------------")
      print(f"The following movie has multiple release years: {movie_name}")
      print("------------------------------")

  else:
    # because the movie_name has a pipe sign |
    # split into two parts => left side is the movie name, right side is the year
    # save to variables
    parts = movie_name.split("|")
    movie_name = parts[0]
    movie_year = parts[1]

    # filter out the movie with same name and year
    search_result = df[df["original_title"] == movie_name]
    search_result_single = search_result[search_result['release_date'].dt.year.astype("string") == movie_year]

    # order search_result_single by popularity, most popular first, ALWAYS CHOOSE FIRST ONE
    search_result_single = search_result_single.sort_values(by='popularity', ascending=False)

    # if we still have multiple movies => choose the most popular one
    search_result_single = search_result_single.iloc[0]

    # save the movie id into variable
    movie_id = search_result_single['id'].astype(int).item()
    movieIds_list.append(movie_id)
    ratings_list.append(movie_rating)
    dates_list.append(movie_watch_date)


#     build the pandas row for a single movie: userId, movieId, rating, date
#     => userId is always 672, movieId we get earlier, rating already in data
#     => simulate the date => on first movie => start from October 2023
#     => after each movie => go one month back


# after we have processed all the movies (after the loop)
# we should have a big dictionary of userId's, movieId's, ratings and dates
print(userIds_list)
print(movieIds_list)
print(ratings_list)
print(dates_list)
# use pandas to convert this big dictionary to a pandas DataFrame

# combine our new DataFrame with the original ratings data

[672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672, 672]
[279, 280, 122, 272, 10313, 120467, 38, 27205, 83542, 18955, 854, 3049, 78, 378018]
[5, 5, 5, 4.5, 4, 4.5, 4, 4, 4.5, 5, 4, 4, 5, 4]
[1668067820, 1666253420, 1664439020, 1662624620, 1660810220, 1658995820, 1657181420, 1655367020, 1653552620, 1651738220, 1649923820, 1648109420, 1646295020, 1644480620]


In [76]:
# get the ratings data from Google Drive
ratings_df = pd.read_csv('data/ratings_small.csv')

# before the code turns timestamps into dates
# inject our own data into the dataframe

new_data_dict = {"userId": userIds_list, "movieId": movieIds_list, "rating": ratings_list, "timestamp": dates_list}
new_rows = pd.DataFrame.from_dict(new_data_dict)

# add our own data into the original ratings_df
ratings_df = pd.concat([ratings_df, new_rows], ignore_index=True)

ratings_df['date'] = ratings_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
ratings_df.drop('timestamp', axis=1, inplace=True)

print(ratings_df.head(10))
print()
print(ratings_df.tail(10))

# things we have to figure out in the ratings_df:
# which user id can we use? (basically, what is biggest id => that + 1)
# biggest id is 671, so our id => 672

# merge the rating data with movie data (df)
# we have to modify the ratings_df before this part
ratings_df = ratings_df.merge(df[['id', 'original_title', 'genres', 'overview']], left_on='movieId',right_on='id', how='left')
ratings_df = ratings_df[~ratings_df['id'].isna()]
ratings_df.drop('id', axis=1, inplace=True)
ratings_df.reset_index(drop=True, inplace=True)

   userId  movieId  rating                date
0       1       31     2.5 2009-12-14 02:52:24
1       1     1029     3.0 2009-12-14 02:52:59
2       1     1061     3.0 2009-12-14 02:53:02
3       1     1129     2.0 2009-12-14 02:53:05
4       1     1172     4.0 2009-12-14 02:53:25
5       1     1263     2.0 2009-12-14 02:52:31
6       1     1287     2.0 2009-12-14 02:53:07
7       1     1293     2.0 2009-12-14 02:52:28
8       1     1339     3.5 2009-12-14 02:52:05
9       1     1343     2.0 2009-12-14 02:52:11

        userId  movieId  rating                date
100008     672    10313     4.0 2022-08-18 08:10:20
100009     672   120467     4.5 2022-07-28 08:10:20
100010     672       38     4.0 2022-07-07 08:10:20
100011     672    27205     4.0 2022-06-16 08:10:20
100012     672    83542     4.5 2022-05-26 08:10:20
100013     672    18955     5.0 2022-05-05 08:10:20
100014     672      854     4.0 2022-04-14 08:10:20
100015     672     3049     4.0 2022-03-24 08:10:20
100016     672

**Continue the original example notebook as it is**

In [77]:
movies_df = df[['id', 'original_title']]
movies_df.rename(columns={'id':'movieId'}, inplace=True)
movies_df.head()

,movieId,original_title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [78]:
ratings_df['userId'] = ratings_df['userId'].astype(str)

ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_df[['userId', 'original_title', 'rating']]))
movies = tf.data.Dataset.from_tensor_slices(dict(movies_df[['original_title']]))

ratings = ratings.map(lambda x: {
    "original_title": x["original_title"],
    "userId": x["userId"],
    "rating": float(x["rating"])
})

movies = movies.map(lambda x: x["original_title"])

In [79]:
total_ratings = len(ratings)

# let's calculate the train/test split => 80% for training, 20% for testing
train_ratings = int(0.8 * total_ratings)
test_ratings = total_ratings - train_ratings

print(train_ratings)
print(test_ratings)

35911
8978


In [80]:
print('Total Data: {}'.format(len(ratings)))

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

# let's fix the numbers to match our new data
train = ratings.take(train_ratings)
test = ratings.skip(train_ratings).take(test_ratings)

Total Data: 44889


In [81]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000).map(lambda x: x["userId"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Movies: {}'.format(len(unique_movie_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Movies: 42373
Unique users: 672


In [82]:
class MovieModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 64

    # User and movie models.
    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128).map(self.movie_model)
        )
    )


    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["userId"])
    # And pick out the movie features and pass them into the movie model.
    movie_embeddings = self.movie_model(features["original_title"])

    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("rating")

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [83]:
model = MovieModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

model.fit(cached_train, epochs=3)

ValueError: Cannot convert '('c', 'o', 'u', 'n', 't', 'e', 'r')' to a shape. Found invalid entry 'c' of type '<class 'str'>'. 

In [ ]:
metrics = model.evaluate(cached_test, return_dict=True)

print(f"\nRetrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}")

In [ ]:
def predict_movie(user, top_n=3):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    # recommends movies out of the entire movies dataset.
    index.index_from_dataset(
      tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))

    print('Top {} recommendations for user {}:\n'.format(top_n, user))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

def predict_rating(user, movie):
    trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
          "userId": np.array([str(user)]),
          "original_title": np.array([movie])
      })
    print("Predicted rating for {}: {}".format(movie, predicted_rating.numpy()[0][0]))

In [ ]:
predict_movie(672, 5)

In [ ]:
# very low score for another LOTR movie...?
# next time: let's try with a smaller dataset for own movies
# and prioritize movies that we like (3.5 or more ratings)
predict_rating(672,'Lord of the Rings: The Two Towers')

In [ ]:
# Get meta data for predicted movie
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(['672']))
pred_movies = pd.DataFrame({'original_title': [i.decode('utf-8') for i in titles[0,:10].numpy()]})

pred_df = pred_movies.merge(ratings_df[['original_title', 'genres', 'overview']], on='original_title', how='left')
pred_df = pred_df[~pred_df['original_title'].duplicated()]
pred_df.reset_index(drop=True, inplace=True)
pred_df.index = np.arange(1, len(pred_df)+1)

pred_df

Overall, this movie recommendation model seems to struggle with a movie taste of various genres and ratings. It also seems the model overfits easily, since the retrieval accuracy really dropped down after we just added 16 movies to the dataset.